<a href="https://colab.research.google.com/github/Marcelo-Maia-Dev/ML-Calc-Imovel/blob/main/preco_imoveis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preço de Imóveis em São Paulo**

Irei treinar um modelo para fazer a previsão do preço de venda de apartamentos na cidade de São Paulo e usar esse modelo para alimentar uma aplicação web mediante deploy.

Como identifiquei as colunas desnecessárias e redundantes, irei direto ao ponto do treinamento, visando mostrar principalmente como exportar e importar o modelo com a biblioteca `joblib`.

## Dados de Imóveis

Os dados usados aqui foram obtidos no [Kaggle](https://www.kaggle.com/datasets/argonalyst/sao-paulo-real-estate-sale-rent-april-2019/download?datasetVersionNumber=1), e foram disponibilizados publicamente pela startup OpenImob.

## Análise e Tratamento dos Dados

Os dados originais contém 13.640 entradas e 16 colunas, sendo a coluna `Price` a nossa variável alvo.

In [1]:
# importar os pacotes necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# importar o dataset para um dataframe
url_dataset = "https://www.dropbox.com/s/h8blgaphkfpqsn5/sao-paulo-properties-april-2019.csv?dl=1"
df = pd.read_csv(url_dataset)

# ver as 5 primeiras entradas
display(df.head())

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,District,Negotiation Type,Property Type,Latitude,Longitude
0,930,220,47,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.543138,-46.479486
1,1000,148,45,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.550239,-46.480718
2,1000,100,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.542818,-46.485665
3,1000,200,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.547171,-46.483014
4,1300,410,55,2,2,1,1,1,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.525025,-46.482436


In [2]:
df_clean = df.copy()

# Limpar os nomes do bairros
df_clean['District'] = df_clean['District'].apply(lambda x: x.split('/')[0])

# ver as 5 primeiras entradas
df_clean.head()

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,District,Negotiation Type,Property Type,Latitude,Longitude
0,930,220,47,2,2,1,1,0,0,0,0,Artur Alvim,rent,apartment,-23.543138,-46.479486
1,1000,148,45,2,2,1,1,0,0,0,0,Artur Alvim,rent,apartment,-23.550239,-46.480718
2,1000,100,48,2,2,1,1,0,0,0,0,Artur Alvim,rent,apartment,-23.542818,-46.485665
3,1000,200,48,2,2,1,1,0,0,0,0,Artur Alvim,rent,apartment,-23.547171,-46.483014
4,1300,410,55,2,2,1,1,1,0,0,0,Artur Alvim,rent,apartment,-23.525025,-46.482436


## Modelo de Machine Learning

Escolhi o modelo Random Forest para treinar meu modelo e observei três principais métricas de avaliação.

In [3]:
# dummy variables
df_clean = pd.get_dummies(df_clean)

# separar entre variáveis X e y
X_simp = df_clean.drop('Price', axis=1)
y_simp = df_clean['Price']

# split entre datasets de treino e teste
X_train_simp, X_test_simp, y_train_simp, y_test_simp = train_test_split(X_simp, y_simp, test_size=0.33)

# instanciar e treinar o modelo
model = RandomForestRegressor(random_state=42)
model.fit(X_train_simp, y_train_simp)

# fazer as previsões em cima do dataset de teste
y_pred_simp = model.predict(X_test_simp)

# métricas de avaliação
print("r2: \t{:.4f}".format(r2_score(y_test_simp, y_pred_simp)))
print("MAE: \t{:.4f}".format(mean_absolute_error(y_test_simp, y_pred_simp)))
print("MSE: \t{:.4f}".format(mean_squared_error(y_test_simp, y_pred_simp)))

r2: 	0.9103
MAE: 	50992.5564
MSE: 	29648792762.9029


#### Salvando o modelo

O nosso modelo está treinado e é capaz de realizar previsões. No entanto, está "preso" ao *kernel* rodando dentro do Google Colab.

Imagine precisar rodar todas as células novamente a cada vez que fosse fazer uma previsão. Seria inviável!

Para conseguir exportar o modelo de *machine learning* (na verdade, isso pode ser feito com qualquer estrutura de dados) vou usar a biblioteca `joblib`.

In [4]:
# salvar o modelo em formato joblib
from joblib import dump, load

dump(model, 'model.joblib') 

['model.joblib']

Uma vez que você exporta o seu modelo, é extremamente importante que você também salve os nomes das *features* que esse modelo espera receber, e tem que ser na ordem exata que ele foi treinado.

Da mesma maneira que fizemos com o modelo, salvei os nomes das variáveis em `features_simples.names`.

In [5]:
# salvar os nomes das features do modelo simples
features = X_train_simp.columns.values

dump(features, 'features.names') 

['features.names']

#### Carregando o modelo

Uma vez que você salvou o modelo em um arquivo, consegue carregar ele novamente usando o `pickle.load()`

In [6]:
# importar modelo e feature names
new_model = load('model.joblib') 
features = load('features.names') 

In [7]:
# ver o tipo da nova variável
type(new_model)

sklearn.ensemble._forest.RandomForestRegressor

In [8]:
import sklearn
sklearn.__version__

'1.2.2'

In [9]:
X_simp

,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,...,District_Vila Maria,District_Vila Mariana,District_Vila Matilde,District_Vila Olimpia,District_Vila Prudente,District_Vila Sônia,District_Água Rasa,Negotiation Type_rent,Negotiation Type_sale,Property Type_apartment
0,220,47,2,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1,148,45,2,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
2,100,48,2,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,200,48,2,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
4,410,55,2,2,1,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13635,420,51,2,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
13636,630,74,3,2,1,2,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1
13637,1100,114,3,3,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1
13638,48,39,1,2,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,1,1


In [10]:
import numpy as np
import json

dict(zip(X_simp.columns.values, np.zeros(X_simp.shape[0]).astype(int)))

{'Condo': 0,
 'Size': 0,
 'Rooms': 0,
 'Toilets': 0,
 'Suites': 0,
 'Parking': 0,
 'Elevator': 0,
 'Furnished': 0,
 'Swimming Pool': 0,
 'New': 0,
 'Latitude': 0,
 'Longitude': 0,
 'District_Alto de Pinheiros': 0,
 'District_Anhanguera': 0,
 'District_Aricanduva': 0,
 'District_Artur Alvim': 0,
 'District_Barra Funda': 0,
 'District_Bela Vista': 0,
 'District_Belém': 0,
 'District_Bom Retiro': 0,
 'District_Brasilândia': 0,
 'District_Brooklin': 0,
 'District_Brás': 0,
 'District_Butantã': 0,
 'District_Cachoeirinha': 0,
 'District_Cambuci': 0,
 'District_Campo Belo': 0,
 'District_Campo Grande': 0,
 'District_Campo Limpo': 0,
 'District_Cangaíba': 0,
 'District_Capão Redondo': 0,
 'District_Carrão': 0,
 'District_Casa Verde': 0,
 'District_Cidade Ademar': 0,
 'District_Cidade Dutra': 0,
 'District_Cidade Líder': 0,
 'District_Cidade Tiradentes': 0,
 'District_Consolação': 0,
 'District_Cursino': 0,
 'District_Ermelino Matarazzo': 0,
 'District_Freguesia do Ó': 0,
 'District_Grajaú': 0

In [11]:
# Serializing json    
json_object = json.dumps(dict(zip(X_simp.columns.values, np.zeros(X_simp.shape[0]).astype(int).tolist())), indent = 4)   
print(json_object)  

{
    "Condo": 0,
    "Size": 0,
    "Rooms": 0,
    "Toilets": 0,
    "Suites": 0,
    "Parking": 0,
    "Elevator": 0,
    "Furnished": 0,
    "Swimming Pool": 0,
    "New": 0,
    "Latitude": 0,
    "Longitude": 0,
    "District_Alto de Pinheiros": 0,
    "District_Anhanguera": 0,
    "District_Aricanduva": 0,
    "District_Artur Alvim": 0,
    "District_Barra Funda": 0,
    "District_Bela Vista": 0,
    "District_Bel\u00e9m": 0,
    "District_Bom Retiro": 0,
    "District_Brasil\u00e2ndia": 0,
    "District_Brooklin": 0,
    "District_Br\u00e1s": 0,
    "District_Butant\u00e3": 0,
    "District_Cachoeirinha": 0,
    "District_Cambuci": 0,
    "District_Campo Belo": 0,
    "District_Campo Grande": 0,
    "District_Campo Limpo": 0,
    "District_Canga\u00edba": 0,
    "District_Cap\u00e3o Redondo": 0,
    "District_Carr\u00e3o": 0,
    "District_Casa Verde": 0,
    "District_Cidade Ademar": 0,
    "District_Cidade Dutra": 0,
    "District_Cidade L\u00edder": 0,
    "District_Cidade T

**Salvar os nomes dos bairros em um arquivo txt**

In [12]:
# Locale Settings
!sudo locale-gen "pt_BR.UTF-8"
!sudo dpkg-reconfigure locales
!export LC_ALL="pt_BR.UTF-8"
!export LC_CTYPE="pt_BR.UTF-8"
!sudo dpkg-reconfigure locales

import locale
locale.setlocale(locale.LC_ALL, '')

Generating locales (this might take a while)...
  pt_BR.UTF-8... done
Generation complete.
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
Configuring locales
-------------------

Locales are a framework to switch between multiple languages and allow users to
use their language, country, characters, collation order, etc.

Please choose which locales to generate. UTF-8 locales should be chosen by
default, particularly for new installations. Other character sets may be useful
for backwards compatibility with older systems and software.

  1. All locales                      252. gl_ES.UTF-8 UTF-8
  2. aa_DJ ISO-8859-1                 253. gl_ES@euro

'en_US.UTF-8'

In [13]:
bairros = [f.split("District_")[1] for f in df_clean.columns if "District_" in f]

In [14]:
dump(sorted(bairros, key=locale.strxfrm), 'bairros.joblib') 

['bairros.joblib']

In [15]:
features

array(['Condo', 'Size', 'Rooms', 'Toilets', 'Suites', 'Parking',
       'Elevator', 'Furnished', 'Swimming Pool', 'New', 'Latitude',
       'Longitude', 'District_Alto de Pinheiros', 'District_Anhanguera',
       'District_Aricanduva', 'District_Artur Alvim',
       'District_Barra Funda', 'District_Bela Vista', 'District_Belém',
       'District_Bom Retiro', 'District_Brasilândia', 'District_Brooklin',
       'District_Brás', 'District_Butantã', 'District_Cachoeirinha',
       'District_Cambuci', 'District_Campo Belo', 'District_Campo Grande',
       'District_Campo Limpo', 'District_Cangaíba',
       'District_Capão Redondo', 'District_Carrão', 'District_Casa Verde',
       'District_Cidade Ademar', 'District_Cidade Dutra',
       'District_Cidade Líder', 'District_Cidade Tiradentes',
       'District_Consolação', 'District_Cursino',
       'District_Ermelino Matarazzo', 'District_Freguesia do Ó',
       'District_Grajaú', 'District_Guaianazes', 'District_Iguatemi',
       'District_